In [ ]:
#Ngoc Tuan HOANG - s147568
#Thesis topic: Heuristic solution approach for the Capacitated Lot-sizing and Scheduling Problem with 
#Sequence-Dependent Setup Times and Costs

#This code is used for the Heuristic 2 presented in the thesis. 

In [1]:
using JuMP          
using Gurobi        

In [2]:
#In this box, three parameters need to be fulfilled. 
#inst: number of the instance being tested
#N: number of products
#T: number of time periods

#Information about instances can be found in the Appendix 2 at the end of the thesis.

inst = 8
Data = readdlm("Instance$(inst).txt")
N=5
T=20

20

In [3]:
#These lines of code are used to read all deterministic parameters (input data). 

d=cell(N,T)
for i=1:N
    for j=1:T
        d[i,j]=Data[i,j]
    end
end
r=cell(N,T)
for i=1:N
    for j=1:T
        r[i,j]=Data[N+i,j]
    end
end
hn=cell(N,T)
for i=1:N
    for j=1:T
        hn[i,j]=Data[2*N+i,j]
    end
end
hr=cell(N,T)
for i=1:N
    for j=1:T
        hr[i,j]=Data[3*N+i,j]
    end
end
an=cell(N,T)
for i=1:N
    for j=1:T
        an[i,j]=Data[4*N+i,j]
    end
end
ar=cell(N,T)
for i=1:N
    for j=1:T
        ar[i,j]=Data[5*N+i,j]
    end
end
p=cell(N,T)
for i=1:N
    for j=1:T
        p[i,j]=Data[6*N+i,j]
    end
end
time=cell(N,N,T)
for t=1:T
    time[:,:,t]=Data[7*N+((t-1)*N+1):7*N+((t-1)*N+N),1:N]
end
cost=cell(N,N,T)
for t=1:T
    cost[:,:,t]=Data[7*N+N*T+((t-1)*N+1):7*N+N*T+((t-1)*N+N),1:N]
end
cap=cell(1,T)
for j=1:T
    cap[1,j]=Data[7*N+2*N*T+1,j]
end
b=cell(N,1)
for i=1:N
    b[i,1]=Data[7*N+2*N*T+1+i,1]
end
upperbound=cell(1,T)
for t=1:T
    upperbound[1,t]=maximum(time[:,:,t])*N
end
jo=Data[8*N+2*N*T+2,1]

3

In [4]:
#The first sub-problem is generated.

#The model and variables are defined.

m=Model(solver=GurobiSolver())
@defVar(m,XN[i=1:N,t=1:T]>=0)
@defVar(m,XR[i=1:N,t=1:T]>=0)
@defVar(m,InvN[i=1:N,t=1:T]>=0)
@defVar(m,InvR[i=1:N,t=1:T]>=0)
@defVar(m,B[i=1:N,t=1:T]>=0)
@defVar(m,Y[i=1:N,t=1:T],Bin)

#Objective function

@setObjective(m,Min,sum{an[i,t]*XN[i,t]+ar[i,t]*XR[i,t]+hn[i,t]*InvN[i,t]+hr[i,t]*InvR[i,t]+
    b[i]*B[i,t],i=1:N,t=1:T})

#Inventory balance constraint

for i=1:N
    for t=2:T             #inventory of finished products
        @addConstraint(m,InvN[i,t]-B[i,t]==InvN[i,t-1]-B[i,t-1]+XN[i,t]+XR[i,t]-d[i,t])  
    end
end
for i=1:N
    for t=1            #inventory of finished products at 1st week 
        @addConstraint(m,InvN[i,t]-B[i,t]==XN[i,t]+XR[i,t]-d[i,t])
    end
end
for i=1:N
    for t=2:T             #inventory of return products
        @addConstraint(m,InvR[i,t]==InvR[i,t-1]+r[i,t]-XR[i,t])
    end
end
for i=1:N
    for t=1            #inventory of return products at 1st week 
        @addConstraint(m,InvR[i,t]==r[i,t]-XR[i,t])
    end
end

#Capacity constraint

for t=1:T       
    @addConstraint(m,sum{p[i,t]*(XN[i,t]+XR[i,t]),i=1:N}<=cap[t]-upperbound[t])
end

#Backorder at the end constraint

for i=1:N       
    for t=T     
        @addConstraint(m,B[i,t]==0)
    end
end

#The relation between variable P and variable X

for i=1:N
    for t=1:T
        @addConstraint(m,XN[i,t]+XR[i,t]<=sum(d)*Y[i,t])    
    end
end

show(m)

Minimization problem with:
 * 325 linear constraints
 * 600 variables: 100 binary
Solver set to Gurobi.Gurobi

In [5]:
#Solving the first sub-problem

tic()
status=solve(m)
toc()

Optimize a model with 325 rows, 600 columns and 1390 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 2e+04]
  Objective range [1e+00, 2e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [3e+01, 2e+04]
Presolve removed 110 rows and 120 columns
Presolve time: 0.02s
Presolved: 215 rows, 480 columns, 1060 nonzeros
Variable types: 480 continuous, 0 integer (0 binary)

Root relaxation: objective 3.337582e+05, 449 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    333758.15773 333758.158  0.00%     -    0s

Explored 0 nodes (449 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 3.337581577298e+05, best bound 3.337581577298e+05, gap 0.0%
elapsed time: 2

2.863981877

.863981877 seconds


In [6]:
#Results of the 1st sub-problem are recorded. 

ObjValue=getObjectiveValue(m)
XNsol=getValue(XN)
XRsol=getValue(XR)

#Some matrices are also prepared for the second sub-problem

Sequence=zeros(T,N)
Obj=cell(1,T)
Bsol=getValue(B)
sum(Bsol)

68.99999999999999

In [7]:
#The second sub-problem is coded in function Salesman.

function Salesman(t)
    
#This code is to find the last product being produced in the previous period, depending on the 
#results of the first sub-problem
if t==1
    jo=Data[8*N+2*N*T+2,1]
    else jo=Sequence[t-1,countnz(Sequence[t-1,:])]
end
    
#Parameter NoP is found   
NoP=0
for i=1:N
    if XNsol[i,t]>0 || XRsol[i,t]>0
        NoP=NoP+1
    end
end

#For each value of t, one sub-problem will be formulated and solved    
#The model and variables are defined.
    
mm=Model(solver=GurobiSolver())
@defVar(mm,Y[i=1:N,j=1:N,s=1:NoP],Bin)

#Objective function
@setObjective(mm,Min,sum{cost[i,j,t]*Y[i,j,s],i=1:N,j=1:N,s=1:NoP})

#Constraint (1)
    
for j=1:N
    for i=1:N
        if j!=jo
            @addConstraint(mm,Y[j,i,1]==0)
        end
    end
end    

#Constraint (2)
    
@addConstraint(mm,sum{Y[jo,i,1],i=1:N}==1)
    
   
#Constraint (3)   
    
for i=1:N
        for s=1:(NoP-1)
             @addConstraint(mm,sum{Y[j,i,s],j=1:N}==sum{Y[i,k,s+1],k=1:N})    
    end
end
    
#Constraint (4) & (5)    
    
for j=1:N
    if XNsol[j,t]>0 || XRsol[j,t]>0
        @addConstraint(mm,sum{Y[i,j,s],i=1:N,s=1:NoP}==1)
    end
    if XNsol[j,t]==0 && XRsol[j,t]==0
        @addConstraint(mm,sum{Y[i,j,s],i=1:N,s=1:NoP}==0)
    end
end

#Solving the sub-problem    
status=solve(mm)

#Recording result of the sub-problem

Ysol=getValue(Y)
sequence=[]
for s=1:NoP
    for i=1:N
        for j=1:N
            if Ysol[i,j,s]==1
                push!(sequence,j)
            end
        end
    end
end
for i=1:NoP
    Sequence[t,i]=sequence[i]
end
value=getObjectiveValue(mm)
Obj[1,t]=value
end

Salesman (generic function with 1 method)

In [9]:
#At the firt time running function Salseman, there will be a warning in red lines about the integer
#variables. We can simply ignore it by run the code a second time.

tic()
for t=1:T
    Salesman(t)
    print(t)
end
toc()

Optimize a model with 46 rows, 125 columns and 350 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e+01, 6e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 184.79
Presolve removed 20 rows and 40 columns
Presolve time: 0.01s
Presolved: 26 rows, 85 columns, 235 nonzeros
Variable types: 0 continuous, 85 integer (85 binary)

Root relaxation: objective 1.769700e+02, 20 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     176.9700000  176.97000  0.00%     -    0s

Explored 0 nodes (20 simplex iterations) in 0.01 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 1.769700000000e+02, best bound 1.769700000000e+02, gap 0.0%
1Optimize a model with 46 rows, 125 columns and 350 nonzeros

0.158003359

Optimize a model with 36 rows, 75 columns and 200 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e+01, 6e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 146.74
Presolve removed 36 rows and 75 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 1.398800000000e+02, best bound 1.398800000000e+02, gap 0.0%
18Optimize a model with 46 rows, 125 columns and 350 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [4e+01, 6e+01]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 206.36
Presolve removed 20 rows and 40 columns
Presolve time: 0.00s
Presolved: 26 rows, 85 columns, 235 nonzeros
Variable types: 0 continuous, 85 integer (85 binary

In [10]:
#This is the sequence for the original problem. The 0.0 can be ignored.

print(Sequence)

[3.0 1.0 5.0 2.0 4.0
 4.0 3.0 2.0 1.0 5.0
 5.0 3.0 4.0 2.0 0.0
 3.0 4.0 5.0 1.0 0.0
 1.0 5.0 3.0 4.0 0.0
 1.0 2.0 3.0 0.0 0.0
 3.0 4.0 5.0 0.0 0.0
 5.0 3.0 1.0 4.0 2.0
 4.0 5.0 0.0 0.0 0.0
 5.0 3.0 1.0 2.0 0.0
 2.0 3.0 1.0 0.0 0.0
 5.0 4.0 2.0 0.0 0.0
 4.0 5.0 3.0 0.0 0.0
 5.0 0.0 0.0 0.0 0.0
 1.0 4.0 0.0 0.0 0.0
 2.0 5.0 1.0 3.0 0.0
 3.0 4.0 2.0 5.0 0.0
 1.0 2.0 3.0 0.0 0.0
 3.0 4.0 1.0 5.0 2.0
 2.0 5.0 1.0 4.0 3.0]

In [11]:
#This box provides the objective value and sum of backorder of the original problem.

println(sum(Obj)+ObjValue)
println(sum(Bsol))

336691.1177298011
68.99999999999999
